<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/fluidflow/FIVcalc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%capture
!pip install neqsim

**Flow-Induced Vibrations (FIV)**

Flow-induced vibrations are dynamic responses of structures when subjected to fluid flow. These vibrations can be caused by various mechanisms, including:

*   **Vortex Shedding:** As fluid flows past a bluff body (an object with a blunt shape), vortices are shed alternately from each side. This shedding creates oscillating forces perpendicular to the flow direction, which can induce vibrations in the body.
*   **Buffeting:** This occurs when turbulent flow imparts random pressure fluctuations on a structure, leading to random vibrations.
*   **Galloping:** This is a self-excited vibration that can occur in structures with non-circular cross-sections, like power lines or bridge decks. It's driven by aerodynamic forces that increase with the amplitude of vibration.
*   **Flutter:** Aeroelastic instability that can occur in flexible structures like aircraft wings. It's a coupling between aerodynamic forces and the structure's elastic response.


# FlowInducedVibrationAnalyser

The `FlowInducedVibrationAnalyser` class is a measurement device within the Neqsim framework designed to assess the likelihood of flow-induced vibration (FIV) in a pipeline. High-velocity turbulent fluids can generate significant kinetic energy, leading to low-frequency vibrations that may damage piping and supports. This class implements methods to quantify the risk of such vibrations.

## Constructors

### FlowInducedVibrationAnalyser(PipeBeggsAndBrills pipe)

Creates a `FlowInducedVibrationAnalyser` instance associated with a specific `PipeBeggsAndBrills` object.

*   **Parameters:**
    *   `pipe`: The pipeline segment to be analyzed.

### FlowInducedVibrationAnalyser(String name, PipeBeggsAndBrills pipe)

Creates a `FlowInducedVibrationAnalyser` instance with a specified name, associated with a `PipeBeggsAndBrills` object.

*   **Parameters:**
    *   `name`: A custom name for the analyzer.
    *   `pipe`: The pipeline segment to be analyzed.

## Methods

### displayResult()

This method is intended to display the analysis results. *Note: This method is currently not implemented.*

### getMeasuredValue(String unit)

Calculates and returns a value representing the likelihood of flow-induced vibration. The calculation method is determined by the `method` property.

*   **Parameters:**
    *   `unit`: The desired unit for the returned value (currently not used).
*   **Returns:**
    *   A `double` representing the calculated Likelihood of Failure (LOF) or Flow-induced Root Mean Square (FRMS) value.

#### Likelihood of Failure (LOF) Method

This method, based on Energy Institute guidelines, calculates a Likelihood of Failure (LOF) score, which is a screening value to assess the risk of vibration-induced fatigue. The LOF is determined by comparing the kinetic energy of the flow to the pipe's resistance to vibration.

The LOF is calculated as follows:

$LOF = \frac{\rho_{mix} \cdot v_{mix}^2 \cdot FVF}{F_v}$

Where:

*   $\rho_{mix}$: Density of the fluid mixture.
*   $v_{mix}$: Velocity of the fluid mixture.
*   $FVF$: A factor accounting for the influence of the gas volume fraction (GVF) on the vibration potential. It is calculated as:
    *   If $GVF \le 0.88$:
        $FVF = 1.0$
    *   If $0.88 < GVF \le 0.99$:
        $FVF = -27.882 \cdot GVF^2 + 45.545 \cdot GVF - 17.495$
    *   If $GVF > 0.99$:
        $FVF = \sqrt{\frac{\mu_{mix}}{\sqrt{0.001}}}$
        Where $\mu_{mix}$ is the mixture viscosity.

*   $F_v$: The flow-induced vibration factor, which represents the pipe's structural resistance to vibration. It is a function of the pipe's geometry and support arrangement. The calculation is:
    $F_v = \alpha \cdot \left(\frac{D_{ext}}{t}\right)^\beta$
    Where:
    *   $D_{ext}$: External diameter of the pipe.
    *   $t$: Pipe wall thickness.
    *   $\alpha$ and $\beta$: Coefficients determined by the `supportArrangement` property. The values are empirically derived and depend on the stiffness of the pipe supports.

| Support Arrangement | $\alpha$ | $\beta$ |
| :--- | :--- | :--- |
| Stiff | $446187 + 646 \cdot D_{ext} + 9.17 \cdot 10^{-4} \cdot D_{ext}^3$ | $0.1 \cdot \ln(D_{ext}) - 1.3739$ |
| Medium stiff | $283921 + 370 \cdot D_{ext}$ | $0.1106 \cdot \ln(D_{ext}) - 1.501$ |
| Medium | $150412 + 209 \cdot D_{ext}$ | $0.0815 \cdot \ln(D_{ext}) - 1.3269$ |
| Flexible | $41.21 \cdot \ln(D_{ext}) + 49397$ | $0.0815 \cdot \ln(D_{ext}) - 1.3842$ |

#### Flow-induced Root Mean Square (FRMS) Method

This method provides an alternative assessment of vibration.

*   If $GVF < 0.8$:
    The method returns the Gas Volume Fraction (GVF).

*   If $GVF \ge 0.8$:
    $FRMS = 1 + 5 \cdot (1 - GVF) \cdot D^{1.6} \cdot C_{FRMS} \cdot \rho_{liq}^{0.6} \cdot v_{mix}^{1.2}$
    Where:
    *   $D$: Pipe diameter.
    *   $C_{FRMS}$: A user-defined constant (`FRMSConstant`).
    *   $\rho_{liq}$: Density of the liquid phase.

### getMethod()

Returns the current calculation method ("LOF" or "FRMS").

*   **Returns:**
    *   A `String` indicating the active method.

### setMethod(String method)

Sets the calculation method to be used.

*   **Parameters:**
    *   `method`: The desired method ("LOF" or "FRMS").

### setSegment(int segment)

Specifies the pipeline segment to be analyzed.

*   **Parameters:**
    *   `segment`: The index of the pipe segment.

### setFRMSConstant(double frms)

Sets the constant used in the FRMS calculation.

*   **Parameters:**
    *   `frms`: The value for the `FRMSConstant`.

### setSupportArrangement(String arrangement)

Defines the type of pipe support arrangement, which influences the $F_v$ calculation.

*   **Parameters:**
    *   `arrangement`: A `String` representing the support stiffness ("Stiff", "Medium stiff", "Medium", or "Flexible").

### setSupportDistance(Double distance)

Sets the distance between pipe supports. *Note: This property is not currently used in the calculations.*

*   **Parameters:**
    *   `distance`: The support distance.

# NeqSim examples

In [11]:
from neqsim import jneqsim

In [50]:
from neqsim.thermo import fluid, TPflash
from neqsim.process import stream

# Create a simple thermodynamic system with methane/ethane
thermoSystem = fluid("srk")
thermoSystem.addComponent("methane", 0.90)
thermoSystem.addComponent("ethane", 0.10)
thermoSystem.addComponent("nC10", 0.10)
thermoSystem.addComponent("water", 1.10)
thermoSystem.setMixingRule("classic")
thermoSystem.setMultiPhaseCheck(True)
thermoSystem.setTemperature(298.15, "K")
thermoSystem.setPressure(70.0, "bara")
thermoSystem.setTotalFlowRate(100000.0, "kg/hr")

# Perform a flash calculation to determine phase properties
TPflash(thermoSystem)
thermoSystem.initPhysicalProperties()

# Create a stream from the thermodynamic system
stream1 = stream('stream 1', thermoSystem)
stream1.setName("test stream")
stream1.run()

# Create a pipe using the Beggs and Brill correlation
pipe = jneqsim.process.equipment.pipeline.PipeBeggsAndBrills('pipe1', stream1)
pipe.setDiameter(0.1)
pipe.setLength(50.0)
pipe.setElevation(0.0)
pipe.setPipeWallRoughness(1.0e-5)
pipe.setThickness(0.01)
pipe.setNumberOfIncrements(10)
pipe.run()

print("Fluid properties and pipe parameters are set up.")
print(f"Density: {thermoSystem.getDensity('kg/m3')} kg/m³")
print(f"Viscosity: {thermoSystem.getViscosity('Pas')} Pa.s")
print(f"Pipe Diameter: {pipe.getDiameter()} m")
print(f"Pipe Length: {pipe.getLength()} m")

analyzerDefaultSegment = jneqsim.process.measurementdevice.FlowInducedVibrationAnalyser("Default segment analyzer", pipe)
analyzerDefaultSegment.setMethod("LOF")
analyzerDefaultSegment.setSupportArrangement("Stiff");
fiv = analyzerDefaultSegment.getMeasuredValue()
print("Stiff segment LOF value: " + str(fiv))
analyzerDefaultSegment.setMethod("FRMS")
frms = analyzerDefaultSegment.getMeasuredValue()
print("Stiff segment FRMS value: " + str(frms))

analyzerDefaultSegment = jneqsim.process.measurementdevice.FlowInducedVibrationAnalyser("Default segment analyzer", pipe)
analyzerDefaultSegment.setMethod("LOF")
analyzerDefaultSegment.setSupportArrangement("Medium");
fiv = analyzerDefaultSegment.getMeasuredValue()
print("Medium segment LOF value: " + str(fiv))
analyzerDefaultSegment.setMethod("FRMS")
frms = analyzerDefaultSegment.getMeasuredValue()
print("Medium segment FRMS value: " + str(frms))

Fluid properties and pipe parameters are set up.
Density: 178.40877112695264 kg/m³
Viscosity: 0.0004862460287701426 Pa.s
Pipe Diameter: 0.1 m
Pipe Length: 50.0 m
Stiff segment LOF value: 1.5382190619225382
Stiff segment FRMS value: 267.5233468338844
Medium segment LOF value: 5.105300996972015
Medium segment FRMS value: 267.5233468338844
